## Quickstart 

In this notebook you will learn how to set up inait's prediction tool and its basic functionalities.

We will use the classic Airline passenger dataset as a companion dataset but feel free to use your own data!

### Prerequisites

Let's import the relevant packages first.

In [ ]:
import sys
import os

sys.path.append(os.path.join(os.path.dirname(os.getcwd())))
import pandas as pd

from inait.prediction_script import predict
from inait.plot_script import plot

Now specify the given base url and authentication key.

In [ ]:
# API Configuration
try:
    base_url = os.environ[
        "API_BASE_URL"
    ]  # Add the URL of your inait forecast deployment/SaaS
    auth_key = os.environ["API_AUTH_KEY"]  # Replace with your actual API key
except KeyError as e:
    raise ValueError(
        f"❌ Missing environment variable: {e}. Please set API_BASE_URL and API_AUTH_KEY."
    )

### Run your first prediction model

We will present an example using the classic Box & Jenkins dataset that contains the monthly number of airline passengers between 1949 and 1960 (source: Box, George EP, et al. Time series analysis: forecasting and control. John Wiley & Sons, 2015.). The goal is to predict the passengers for the next year, i.e., 12 steps ahead. 

To forecast using inait models you need to specifiy the dataset using `data_path`, how many steps in the future you want to forecast (the so-called `forecasting_horizon`), and how many steps in the past you want to use to make a prediction (`observation_length`). 

In [ ]:
### Load the data
data_path = "../data/Airline.csv"
data = pd.read_csv(
    data_path, index_col=0
)  # dataset must have a valid datetime index with fixed frequency

# Configure prediction parameters
target_columns = [
    "number_of_passengers"
]  # List of target columns to predict in the dataset
forecasting_horizon = 12  # Predict 12 time steps ahead
observation_length = 24  # Use last 24 time steps as historical context

In [ ]:
prediction, session_id = predict(
    base_url=base_url,
    auth_key=auth_key,
    data_path=data_path,
    target_columns=target_columns,
    forecasting_horizon=forecasting_horizon,
    observation_length=observation_length,
)

prediction

In [ ]:
plot(historical_data=data, predicted_data=prediction)

You can now try it out with different `forecasting_horizon` or `observation_length` parameters, or use you own dataset!

In [ ]:
from inait.explanation_script import create_explanation_payload
from inait.utils import make_request

payload = create_explanation_payload(
    session_id=session_id, cutoff_days="1960-12-31 00:00:00"
)
explanation_response = make_request(
    base_url + "/explanation", payload, auth_key=auth_key
)
df = explanation_response["response"]["data"]["1960-12-31 00:00:00"][
    "1961-01-31 00:00:00"
]

df = pd.DataFrame([df])
df = df.iloc[:, :-1]
df = df.T

df